In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.CropNumpy import CropNc2Np
import joblib

In [2]:
nc_path = r'data\dataset\xrain\raw_nc\202207172200_XRAIN_10min.nc'
dr = xr.open_dataset(nc_path)['rainfall']
dr

<xarray.DataArray 'rainfall' (time: 229, y: 1480, x: 1178)> Size: 2GB
[399247760 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 2kB 2022-07-17T22:00:00 ... 2022-07-19T1...
    latitude   (y) float64 12kB ...
    longitude  (x) float64 9kB ...
Dimensions without coordinates: y, x

In [3]:
import xarray as xr
import numpy as np
from scipy.signal import convolve2d


# 2. 定义 8x8 的平均卷积核
kernel = np.ones((8, 8)) / 64

# 3. 创建一个空列表来存放每个时间步长处理后的结果
smoothed_data_list = []

# 4. 遍历时间维度
# 我们使用 for 循环遍历 dr 中的每一个时间切片
for time_step in dr['time']:
    # 提取当前时间步长的数据，它是一个二维数组
    dr_2d = dr.sel(time=time_step).data

    # 对这个二维数组进行卷积
    smoothed_2d = convolve2d(
        dr_2d,
        kernel,
        mode='same',
        boundary='fill',
        fillvalue=0
    )

    # 将处理后的二维数组添加到列表中
    smoothed_data_list.append(smoothed_2d)

# 5. 将处理后的结果重新组合成一个新的三维 xarray.DataArray
# stack() 函数可以将多个数组沿着新的维度堆叠起来
smoothed_dr = xr.DataArray(
    np.stack(smoothed_data_list),
    dims=dr.dims,
    coords=dr.coords
)

print("原始数据维度:", dr.sizes)
print("处理后数据维度:", smoothed_dr.sizes)
# smoothed_dr = smoothed_dr.rename({'__xarray_dataarray_variable__': 'rainfall'})
# smoothed_dr.to_netcdf(r'data\dataset\xrain\raw_nc\smoothed_raw_xrain.nc')

原始数据维度: Frozen({'time': 229, 'y': 1480, 'x': 1178})
处理后数据维度: Frozen({'time': 229, 'y': 1480, 'x': 1178})


In [8]:
joblib.dump(np.array(CropNc2Np(r'data\dataset\xrain\raw_nc\xrain_smooth.nc',slide_T=20,slide_H=64,slide_W=64)), r"data\dataset\xrain\figure\smoothed_xrain.joblib")

3300 crops can be extracted.


Cropping Time Slices: 100%|██████████| 11/11 [00:00<00:00, 10995.55it/s]


['data\\dataset\\xrain\\figure\\smoothed_xrain.joblib']